In [1]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Read Data
data = pd.read_csv('metadata.csv')
data.head(3)

,sr,audio,classID,fold
0,0,LJ001-0001.wav,1,11
1,1,LJ001-0002.wav,1,11
2,2,LJ001-0003.wav,1,11


In [3]:
data.shape

(20522, 4)

In [4]:
valid_data = data[['audio', 'classID', 'fold']]
valid_data.shape

(20522, 3)

In [5]:
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['audio'].astype('str')

In [6]:
audio_files = []

for row in valid_data.itertuples():
    audio_files.append(row.path)

In [7]:
D = [] # Dataset

for row, row_dict in zip(audio_files, valid_data.classID):
    try:
        y, sr = librosa.load('dataset/audio/'+row, duration=2.97)
        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        if ps.shape != (128, 128): continue
        D.append( (ps, row_dict) )
    except:
        print('Something is wrong with '+row)
    else:
        continue

Something is wrong with fold9/101729-0-0-1.wav
Something is wrong with fold9/101729-0-0-11.wav
Something is wrong with fold9/101729-0-0-12.wav
Something is wrong with fold9/101729-0-0-13.wav
Something is wrong with fold9/101729-0-0-14.wav
Something is wrong with fold9/101729-0-0-16.wav
Something is wrong with fold9/101729-0-0-17.wav
Something is wrong with fold9/101729-0-0-18.wav
Something is wrong with fold9/101729-0-0-19.wav
Something is wrong with fold9/101729-0-0-21.wav
Something is wrong with fold9/101729-0-0-22.wav
Something is wrong with fold9/101729-0-0-23.wav
Something is wrong with fold9/101729-0-0-24.wav
Something is wrong with fold9/101729-0-0-26.wav
Something is wrong with fold9/101729-0-0-28.wav
Something is wrong with fold9/101729-0-0-29.wav
Something is wrong with fold9/101729-0-0-3.wav
Something is wrong with fold9/101729-0-0-32.wav
Something is wrong with fold9/101729-0-0-33.wav
Something is wrong with fold9/101729-0-0-36.wav
Something is wrong with fold9/101729-0-0-3

Something is wrong with fold2/113201-5-0-5.wav
Something is wrong with fold2/113201-5-0-6.wav
Something is wrong with fold2/113201-5-0-7.wav
Something is wrong with fold8/113202-5-0-0.wav
Something is wrong with fold8/113202-5-0-1.wav
Something is wrong with fold8/113202-5-0-10.wav
Something is wrong with fold8/113202-5-0-11.wav
Something is wrong with fold8/113202-5-0-12.wav
Something is wrong with fold8/113202-5-0-13.wav
Something is wrong with fold8/113202-5-0-14.wav
Something is wrong with fold8/113202-5-0-15.wav
Something is wrong with fold8/113202-5-0-16.wav
Something is wrong with fold8/113202-5-0-17.wav
Something is wrong with fold8/113202-5-0-18.wav
Something is wrong with fold8/113202-5-0-19.wav
Something is wrong with fold8/113202-5-0-2.wav
Something is wrong with fold8/113202-5-0-20.wav
Something is wrong with fold8/113202-5-0-21.wav
Something is wrong with fold8/113202-5-0-22.wav
Something is wrong with fold8/113202-5-0-23.wav
Something is wrong with fold8/113202-5-0-24.wa

Something is wrong with fold3/132855-2-0-90.wav
Something is wrong with fold3/132855-2-0-99.wav
Something is wrong with fold6/133494-2-0-18.wav
Something is wrong with fold6/133494-2-0-19.wav
Something is wrong with fold6/133494-2-0-22.wav
Something is wrong with fold6/133494-2-0-25.wav
Something is wrong with fold6/133494-2-0-26.wav
Something is wrong with fold6/133494-2-0-31.wav
Something is wrong with fold6/133494-2-0-37.wav
Something is wrong with fold6/133494-2-0-45.wav
Something is wrong with fold6/133797-6-0-0.wav
Something is wrong with fold6/133797-6-1-0.wav
Something is wrong with fold6/133797-6-2-0.wav
Something is wrong with fold1/135776-2-0-32.wav
Something is wrong with fold1/135776-2-0-37.wav
Something is wrong with fold1/135776-2-0-40.wav
Something is wrong with fold1/135776-2-0-49.wav
Something is wrong with fold1/135776-2-0-50.wav
Something is wrong with fold1/135776-2-0-65.wav
Something is wrong with fold1/135776-2-0-85.wav
Something is wrong with fold1/135776-2-0-90

Something is wrong with fold4/146709-0-0-1.wav
Something is wrong with fold4/146709-0-0-11.wav
Something is wrong with fold4/146709-0-0-12.wav
Something is wrong with fold4/146709-0-0-17.wav
Something is wrong with fold4/146709-0-0-18.wav
Something is wrong with fold4/146709-0-0-19.wav
Something is wrong with fold4/146709-0-0-20.wav
Something is wrong with fold4/146709-0-0-22.wav
Something is wrong with fold4/146709-0-0-25.wav
Something is wrong with fold4/146709-0-0-26.wav
Something is wrong with fold4/146709-0-0-3.wav
Something is wrong with fold4/146709-0-0-30.wav
Something is wrong with fold4/146709-0-0-33.wav
Something is wrong with fold4/146709-0-0-34.wav
Something is wrong with fold4/146709-0-0-37.wav
Something is wrong with fold4/146709-0-0-44.wav
Something is wrong with fold4/146709-0-0-48.wav
Something is wrong with fold4/146709-0-0-53.wav
Something is wrong with fold4/146709-0-0-59.wav
Something is wrong with fold4/146709-0-0-63.wav
Something is wrong with fold4/146709-0-0-6

Something is wrong with fold2/152908-5-0-3.wav
Something is wrong with fold2/152908-5-0-4.wav
Something is wrong with fold2/152908-5-0-5.wav
Something is wrong with fold2/152908-5-0-6.wav
Something is wrong with fold2/152908-5-0-7.wav
Something is wrong with fold2/152908-5-0-8.wav
Something is wrong with fold2/152908-5-0-9.wav
Something is wrong with fold9/155044-9-0-11.wav
Something is wrong with fold9/155044-9-0-15.wav
Something is wrong with fold9/155044-9-0-19.wav
Something is wrong with fold9/155044-9-0-3.wav
Something is wrong with fold9/155044-9-0-37.wav
Something is wrong with fold9/155044-9-0-38.wav
Something is wrong with fold6/155127-9-0-2.wav
Something is wrong with fold6/155127-9-1-2.wav
Something is wrong with fold6/155127-9-1-23.wav
Something is wrong with fold6/155127-9-1-24.wav
Something is wrong with fold6/155127-9-1-25.wav
Something is wrong with fold6/155127-9-1-27.wav
Something is wrong with fold2/155129-1-0-0.wav
Something is wrong with fold2/155129-1-1-0.wav
Some

...

Something is wrong with fold3/185373-9-0-21.wav
Something is wrong with fold3/185373-9-0-6.wav
Something is wrong with fold3/185373-9-1-17.wav
Something is wrong with fold3/185373-9-1-46.wav
Something is wrong with fold3/185373-9-1-7.wav
Something is wrong with fold3/185373-9-1-70.wav
Something is wrong with fold9/185374-9-0-16.wav
Something is wrong with fold9/185374-9-0-18.wav
Something is wrong with fold9/185374-9-0-23.wav
Something is wrong with fold9/185374-9-0-30.wav
Something is wrong with fold9/185374-9-0-33.wav
Something is wrong with fold9/185374-9-0-35.wav
Something is wrong with fold9/185374-9-0-60.wav
Something is wrong with fold3/185375-9-0-23.wav
Something is wrong with fold3/185375-9-0-26.wav
Something is wrong with fold3/185375-9-0-33.wav
Something is wrong with fold3/185375-9-0-60.wav
Something is wrong with fold3/185375-9-0-61.wav
Something is wrong with fold3/185375-9-0-84.wav
Something is wrong with fold4/187356-1-0-0.wav
Something is wrong with fold4/187356-1-1-0.

Something is wrong with fold2/194841-9-0-164.wav
Something is wrong with fold2/194841-9-0-178.wav
Something is wrong with fold2/194841-9-0-222.wav
Something is wrong with fold2/194841-9-0-48.wav
Something is wrong with fold5/194910-9-0-108.wav
Something is wrong with fold5/194910-9-0-26.wav
Something is wrong with fold5/194910-9-0-5.wav
Something is wrong with fold5/194910-9-0-63.wav
Something is wrong with fold5/194910-9-0-65.wav
Something is wrong with fold4/195969-0-0-1.wav
Something is wrong with fold4/195969-0-0-10.wav
Something is wrong with fold4/195969-0-0-12.wav
Something is wrong with fold4/195969-0-0-13.wav
Something is wrong with fold4/195969-0-0-14.wav
Something is wrong with fold4/195969-0-0-15.wav
Something is wrong with fold4/195969-0-0-17.wav
Something is wrong with fold4/195969-0-0-18.wav
Something is wrong with fold4/195969-0-0-19.wav
Something is wrong with fold4/195969-0-0-2.wav
Something is wrong with fold4/195969-0-0-20.wav
Something is wrong with fold4/195969-0-

Something is wrong with fold2/203929-7-3-2.wav
Something is wrong with fold2/203929-7-3-3.wav
Something is wrong with fold2/203929-7-3-5.wav
Something is wrong with fold2/203929-7-3-9.wav
Something is wrong with fold2/203929-7-4-0.wav
Something is wrong with fold2/203929-7-4-1.wav
Something is wrong with fold2/203929-7-4-10.wav
Something is wrong with fold2/203929-7-4-11.wav
Something is wrong with fold2/203929-7-4-12.wav
Something is wrong with fold2/203929-7-4-2.wav
Something is wrong with fold2/203929-7-4-4.wav
Something is wrong with fold2/203929-7-4-6.wav
Something is wrong with fold2/203929-7-4-7.wav
Something is wrong with fold2/203929-7-4-8.wav
Something is wrong with fold2/203929-7-5-0.wav
Something is wrong with fold2/203929-7-5-1.wav
Something is wrong with fold2/203929-7-5-10.wav
Something is wrong with fold2/203929-7-5-11.wav
Something is wrong with fold2/203929-7-5-6.wav
Something is wrong with fold2/203929-7-5-7.wav
Something is wrong with fold2/203929-7-6-1.wav
Somethin

Something is wrong with fold7/34871-3-11-0.wav
Something is wrong with fold7/34871-3-6-0.wav
Something is wrong with fold7/34871-3-8-0.wav
Something is wrong with fold7/34871-3-9-0.wav
Something is wrong with fold5/34872-3-0-0.wav
Something is wrong with fold5/34872-3-0-1.wav
Something is wrong with fold5/34872-3-0-2.wav
Something is wrong with fold6/35548-9-0-10.wav
Something is wrong with fold6/35548-9-0-21.wav
Something is wrong with fold6/35548-9-0-23.wav
Something is wrong with fold6/35548-9-1-14.wav
Something is wrong with fold6/35548-9-2-7.wav
Something is wrong with fold6/35548-9-2-9.wav
Something is wrong with fold8/36429-2-0-13.wav
Something is wrong with fold8/36429-2-0-14.wav
Something is wrong with fold8/36429-2-0-15.wav
Something is wrong with fold8/36429-2-0-18.wav
Something is wrong with fold8/36429-2-0-23.wav
Something is wrong with fold8/36429-2-0-6.wav
Something is wrong with fold8/36429-2-0-7.wav
Something is wrong with fold3/37560-4-0-0.wav
Something is wrong with 

Something is wrong with fold5/76566-3-0-11.wav
Something is wrong with fold5/76566-3-0-3.wav
Something is wrong with fold5/76566-3-0-6.wav
Something is wrong with fold5/76566-3-0-8.wav
Something is wrong with fold4/76568-9-0-10.wav
Something is wrong with fold4/76568-9-0-11.wav
Something is wrong with fold4/76568-9-0-12.wav
Something is wrong with fold4/76568-9-0-13.wav
Something is wrong with fold4/76568-9-0-14.wav
Something is wrong with fold4/76568-9-0-2.wav
Something is wrong with fold4/76585-9-0-10.wav
Something is wrong with fold4/76585-9-0-38.wav
Something is wrong with fold4/76585-9-0-5.wav
Something is wrong with fold4/76585-9-0-57.wav
Something is wrong with fold4/76585-9-0-60.wav
Something is wrong with fold4/76585-9-0-69.wav
Something is wrong with fold6/78651-5-0-0.wav
Something is wrong with fold6/78651-5-0-1.wav
Something is wrong with fold6/78651-5-0-10.wav
Something is wrong with fold6/78651-5-0-11.wav
Something is wrong with fold6/78651-5-0-12.wav
Something is wrong w

In [8]:
print("Number of samples: ", len(D))

Number of samples:  15929


In [10]:
dataset = D
random.shuffle(dataset)

train = dataset[:14500]
test = dataset[14500:]

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(keras.utils.to_categorical(y_train, 4))
y_test = np.array(keras.utils.to_categorical(y_test, 4))

In [13]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(4))
model.add(Activation('softmax'))

In [14]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=12,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 14500 samples, validate on 1429 samples
Epoch 1/12
14500/14500 [==============================] - 188s 13ms/step - loss: 0.4646 - acc: 0.9337 - val_loss: 0.1472 - val_acc: 0.9825
Epoch 2/12
14500/14500 [==============================] - 182s 13ms/step - loss: 0.1261 - acc: 0.9841 - val_loss: 0.0660 - val_acc: 0.9895
Epoch 3/12
14500/14500 [==============================] - 182s 13ms/step - loss: 0.0596 - acc: 0.9911 - val_loss: 0.0663 - val_acc: 0.9895
Epoch 4/12
14500/14500 [==============================] - 182s 13ms/step - loss: 0.0431 - acc: 0.9939 - val_loss: 0.0480 - val_acc: 0.9923
Epoch 5/12
14500/14500 [==============================] - 181s 12ms/step - loss: 0.0381 - acc: 0.9941 - val_loss: 0.0373 - val_acc: 0.9951
Epoch 6/12
14500/14500 [==============================] - 181s 12ms/step - loss: 0.0210 - acc: 0.9968 - val_loss: 0.0556 - val_acc: 0.9902
Epoch 7/12
14500/14500 [==============================] - 182s 13ms/s

In [15]:
model.save('voice_model.h5')